In [ ]:
!pip install tensorflow==2.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 32.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3
  Attempting unin

In [ ]:
%%capture
!apt update
!pip install kaggle

In [ ]:
# # Run this cell to mount your drive (you will be prompted to sign in)
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Create the kaggle directory and 

# (NOTE: Do NOT run this cell more than once unless restarting kernel)
!mkdir ~/.kaggle

In [ ]:
# Read the uploaded kaggle.json file
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
# Download dataset
!!kaggle competitions download -c cassava-leaf-disease-classification

['Traceback (most recent call last):',
 '  File "/usr/local/bin/kaggle", line 5, in <module>',
 '    from kaggle.cli import main',
 '  File "/usr/local/lib/python3.9/dist-packages/kaggle/__init__.py", line 23, in <module>',
 '    api.authenticate()',
 '  File "/usr/local/lib/python3.9/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate',
 "    raise IOError('Could not find {}. Make sure it\\'s located in'",
 "OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method."]

In [ ]:
# Unzip folder in Colab content folder
!unzip cassava-leaf-disease-classification.zip

unzip:  cannot find or open cassava-leaf-disease-classification.zip, cassava-leaf-disease-classification.zip.zip or cassava-leaf-disease-classification.zip.ZIP.


In [ ]:
import glob, random, os, warnings
import time
import shutil
import pathlib
import itertools

# import data handling tools
# import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.layers as L
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

modules loaded


In [ ]:
def seed_everything(seed = 0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed_everything()
warnings.filterwarnings('ignore')

In [ ]:
#Data clean up reference: https://www.kaggle.com/code/abdallahwagih/cassava-leaf-diseases-detection-competition#Model-Structure

In [ ]:
def define_paths(data_dir, csv_dir):
    filepaths = []
    labels = []
    
    df = pd.read_csv(csv_dir)
    folds = os.listdir(data_dir)
    
    for file, i in zip(sorted(folds), df['label']):
            fpath = os.path.join(data_dir, file)
            filepaths.append(fpath)
            if i == 0:    
                labels.append('Cassava Bacterial Blight (CBB)')
            elif i == 1:
                labels.append('Cassava Brown Streak Disease (CBSD)')
            elif i == 2:
                labels.append('Cassava Green Mottle (CGM)')
            elif i == 3:
                labels.append('Cassava Mosaic Disease (CMD)')
            elif i == 4:
                labels.append('Healthy')

    return filepaths, labels


# Concatenate data paths with labels into one dataframe ( to later be fitted into the model )
def define_df(files, classes):
    Fseries = pd.Series(files, name= 'filepaths')
    Lseries = pd.Series(classes, name='labels')
    return pd.concat([Fseries, Lseries], axis= 1)

def create_df(tr_dir, csv_dir):
    # train and valid dataframe
    files, classes = define_paths(tr_dir, csv_dir)
    df = define_df(files, classes)
    strat = df['labels']
    train_df, valid_df = train_test_split(df, train_size= 0.8, shuffle= True, random_state= 123, stratify= strat)

    return train_df, valid_df

In [ ]:
def create_gens(train_df, valid_df, batch_size):
    '''
    This function takes train, validation, and test dataframe and fit them into image data generator, because model takes data from image data generator.
    Image data generator converts images into tensors. '''

    # define model parameters
    img_size = (224, 224)
    channels = 3 # either BGR or Grayscale
    color = 'rgb'
    img_shape = (img_size[0], img_size[1], channels)

    # This function which will be used in image data generator for data augmentation, it just take the image and return it again.
    def scalar(img):
        return img

    # without augmentation
    # tr_gen = ImageDataGenerator(preprocessing_function= scalar, horizontal_flip= True)

    # with data augmentation
    tr_gen = ImageDataGenerator(
      preprocessing_function=scalar,
      horizontal_flip=True,
      rotation_range=15,  # Rotate images up to 15 degrees
      zoom_range=0.1,  # Zoom in/out images up to 20%
      width_shift_range=0.1,  # Shift images horizontally up to 20% of the width
      height_shift_range=0.1,  # Shift images vertically up to 20% of the height
      shear_range=0.2,
      fill_mode='nearest',
      channel_shift_range=50
    )
    ts_gen = ImageDataGenerator(preprocessing_function= scalar)

    train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= color, shuffle= True, batch_size= batch_size)

    valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= color, shuffle= True, batch_size= batch_size)

    return train_gen, valid_gen

In [ ]:
def trim (df, max_size, min_size, column):
    df = df.copy()
    original_class_count = len(list(df[column].unique()))
    print ('Original Number of classes in dataframe: ', original_class_count)
    
    sample_list = [] 
    groups = df.groupby(column)
    
    for label in df[column].unique():        
        group = groups.get_group(label)
        sample_count = len(group)     
        if sample_count > max_size :
            strat = group[column]
            samples, _ = train_test_split(group, train_size= max_size, shuffle= True, 
                                          random_state= 123, stratify= strat)            
            sample_list.append(samples)
      
        elif sample_count >= min_size:
            sample_list.append(group)
    
    df = pd.concat(sample_list, axis= 0).reset_index(drop= True)
    final_class_count = len(list(df[column].unique())) 
    if final_class_count != original_class_count:
        print ('*** WARNING***  dataframe has a reduced number of classes' )
    balance = list(df[column].value_counts())
    print (balance)
    return df

In [ ]:
def show_images(gen):
    '''
    This function take the data generator and show sample of the images
    '''

    # return classes , images to be displayed
    g_dict = gen.class_indices        # defines dictionary {'class': index}
    classes = list(g_dict.keys())     # defines list of dictionary's kays (classes), classes names : string
    images, labels = next(gen)        # get a batch size samples from the generator

    # calculate number of displayed samples
    length = len(labels)        # length of batch size
    sample = min(length, 25)    # check if sample less than 25 images

    plt.figure(figsize= (20, 20))

    # for i in range(sample):
    #     plt.subplot(5, 5, i + 1)
    #     image = images[i] / 255       # scales data to range (0 - 255)
    #     plt.imshow(image)
    #     index = np.argmax(labels[i])  # get image index
    #     class_name = classes[index]   # get class of image
    #     plt.title(class_name, color= 'blue', fontsize= 12)
    #     plt.axis('off')
    for i in range(25):  # Display 25 augmented images
        plt.subplot(5, 5, i + 1)
        images, labels = next(gen)    # get a batch of samples from the generator
        image = images[0] / 255        # scales data to range (0 - 255)
        plt.imshow(image)
        index = np.argmax(labels[0])  # get image index
        class_name = classes[index]   # get class of image
        plt.title(class_name, color='blue', fontsize=12)
        plt.axis('off')
    plt.show()

In [ ]:
# This seems redundant
# def create_gens(train_df, valid_df, batch_size):
#     '''
#     This function takes train, validation, and test dataframe and fit them into image data generator, because model takes data from image data generator.
#     Image data generator converts images into tensors. '''

#     # define model parameters
#     img_size = (224, 224)
#     channels = 3 # either BGR or Grayscale
#     color = 'rgb'
#     img_shape = (img_size[0], img_size[1], channels)

#     # This function which will be used in image data generator for data augmentation, it just take the image and return it again.
#     def scalar(img):
#         return img

#     tr_gen = ImageDataGenerator(preprocessing_function= scalar, horizontal_flip= True)
#     ts_gen = ImageDataGenerator(preprocessing_function= scalar)

#     train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
#                                         color_mode= color, shuffle= True, batch_size= batch_size)

#     valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
#                                         color_mode= color, shuffle= True, batch_size= batch_size)

#     return train_gen, valid_gen

In [20]:
batch_size = 64
n_classes = 5

train_dir = '/content/drive/MyDrive/cassava-leaf-disease-classification/train_images'
csv_dir = '/content/drive/MyDrive/cassava-leaf-disease-classification/train.csv'

# Get splitted data
train_df, valid_df = create_df(train_dir, csv_dir)
train_df = trim(train_df, max_size = 2000, min_size = 1, column='labels')

# Print out the number of samples per class
# class_counts = train_df['labels'].value_counts()
# print(class_counts)

# Get Generators
train_gen, valid_gen = create_gens(train_df, valid_df, batch_size)

Original Number of classes in dataframe:  5
[2000, 2000, 1909, 1751, 870]
Found 8530 validated image filenames belonging to 5 classes.
Found 4280 validated image filenames belonging to 5 classes.


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# Generator for test set
test_dir = '/content/drive/MyDrive/cassava-leaf-disease-classification/test_images'
test_gen = tf.keras.utils.image_dataset_from_directory(test_dir, labels=None, label_mode=None, color_mode='rgb',
                                                        image_size=(224,224), shuffle=False)

Found 1 files belonging to 1 classes.


In [ ]:
#pd.DataFrame(train_df)

show_images(train_gen)

In [ ]:
# #run this once and then comment out 

# from google.colab import files
# train_df.to_csv('train_df.csv') 
# files.download('train_df.csv')

# valid_df.to_csv('valid_df.csv') 
# files.download('valid_df.csv')

## Vision Transformer

In [ ]:
# Model hyperparameters
learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 1

patch_size = 7  # Size of the patches to be extract from the input images
num_patches = (224 // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [56, 28]  # Size of the dense layers of the final classifier

In [ ]:
# Building Multilayer Perceptron (MLP)
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = L.Dense(units, activation = tf.nn.gelu)(x)
        x = L.Dropout(dropout_rate)(x)
    return x

In [ ]:
# Patch Creation Layer
class Patches(L.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = 'VALID',
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
# Sample image patches visualization
plt.figure(figsize=(4, 4))

x = train_gen.next()
image = x[0][0]

plt.imshow(image.astype('uint8'))
plt.axis('off')

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size = (image_size, image_size)
)

patches = Patches(patch_size)(resized_image)
print(f'Image size: {image_size} X {image_size}')
print(f'Patch size: {patch_size} X {patch_size}')
print(f'Patches per image: {patches.shape[1]}')
print(f'Elements per patch: {patches.shape[-1]}')

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))

for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype('uint8'))
    plt.axis('off')

In [ ]:
# Patch Encoding Layer

# The PatchEncoder layer will linearly transform a patch by projecting it into a vector of size projection_dim. 
# In addition, it adds a learnable position embedding to the projected vector.
class PatchEncoder(L.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = L.Dense(units = projection_dim)
        self.position_embedding = L.Embedding(
            input_dim = num_patches, output_dim = projection_dim
        )

    def call(self, patch):
        positions = tf.range(start = 0, limit = self.num_patches, delta = 1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
# Building ViT model
def vision_transformer():
    inputs = L.Input(shape = (image_size, image_size, 3))
    
    # Create patches.
    patches = Patches(patch_size)(inputs)
    
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        
        # Layer normalization 1.
        x1 = L.LayerNormalization(epsilon = 1e-6)(encoded_patches)
        
        # Create a multi-head attention layer.
        attention_output = L.MultiHeadAttention(
            num_heads = num_heads, key_dim = projection_dim, dropout = 0.1
        )(x1, x1)
        
        # Skip connection 1.
        x2 = L.Add()([attention_output, encoded_patches])
        
        # Layer normalization 2.
        x3 = L.LayerNormalization(epsilon = 1e-6)(x2)
        
        # MLP.
        x3 = mlp(x3, hidden_units = transformer_units, dropout_rate = 0.1)
        
        # Skip connection 2.
        encoded_patches = L.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = L.LayerNormalization(epsilon = 1e-6)(encoded_patches)
    representation = L.Flatten()(representation)
    representation = L.Dropout(0.5)(representation)
    
    # Add MLP.
    features = mlp(representation, hidden_units = mlp_head_units, dropout_rate = 0.5)
    
    # Classify outputs.
    logits = L.Dense(n_classes)(features)
    
    # Create the model.
    model = tf.keras.Model(inputs = inputs, outputs = logits)
    
    return model

In [ ]:
decay_steps = train_gen.n // train_gen.batch_size
initial_learning_rate = learning_rate

lr_decayed_fn = tf.keras.experimental.CosineDecay(initial_learning_rate, decay_steps)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_decayed_fn)

In [ ]:
image_size = 224
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

model = vision_transformer()
    
model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.1), 
              metrics = ['accuracy'])


STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = valid_gen.n // valid_gen.batch_size

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 5,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = './model.hdf5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

callbacks = [earlystopping, lr_scheduler, checkpointer]

model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = valid_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = num_epochs,
          callbacks = callbacks)

In [ ]:
# Results
print('Training results')
model.evaluate(train_gen)

print('Validation results')
model.evaluate(valid_gen)

# ResNet50

In [24]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

def create_resnet50_model(input_shape, num_classes):
    # Load pre-trained ResNet50 without the top layer (include_top=False)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Add custom layers for classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Create the final model
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Define the input shape and number of classes for the Cassava Leaf Disease dataset

input_shape = (224, 224, 3)
num_classes = 5

# Create the ResNet50 model
resnet50_model = create_resnet50_model(input_shape, num_classes)

# Compile the model
resnet50_model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [25]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = valid_gen.n // valid_gen.batch_size

In [26]:
#train the data 

resnet50_model.fit(
    train_gen,
    steps_per_epoch= STEP_SIZE_TRAIN,
    epochs=10,
    validation_data=valid_gen
)

Epoch 1/10


KeyboardInterrupt: ignored

In [ ]:
print('Training results')
resnet50_model.evaluate(train_gen)

print('Validation results')
resnet50_model.evaluate(valid_gen)

Training results
  1/134 [..............................] - ETA: 39:50 - loss: 1.9417 - accuracy: 0.2031